In [ ]:
"""
ДЗ 1. Блок 1. Введение в теорию вероятностей. 0.1 курс на Stepik. Метод МонтеКарло.

ч.3 .
"""

In [ ]:
# pylint: disable=unused-import

from random import shuffle

# доп import чтоб линтеры проходили
# from typing import List, Optional, Tuple, Union
from typing import List, Tuple

import numpy as np
import pandas as pd

In [3]:
# Парадокс ДР
# Определить вероятность того, что в группе, состоящей из 23 человек,
# у двух людей будет совпадение дней рождения (число и месяц).


bd = pd.Series(range(365))

In [5]:
t_data = bd.sample(23, replace=True)

t_data.duplicated()

349    False
159    False
176    False
241    False
129    False
220    False
220     True
227    False
261    False
33     False
26     False
291    False
152    False
317    False
108    False
40     False
209    False
340    False
314    False
353    False
9      False
2      False
197    False
dtype: bool

In [6]:
rooms = [bd.sample(23, replace=True).duplicated().max() for _ in range(10000)]

In [10]:
np.mean(rooms)

0.5049

In [ ]:
# Парадокс 2. Экзамент и билеты
# Экзамен проходит по следующей схеме: если некоторый билет уже был вытянут,
# то после ответа экзаменатор откладывает его в сторону.
# Студент выучил 20 билетов из 30. Когда ему выгоднее идти, первым или вторым,
# чтобы вероятность вынуть выученный билет была больше?

In [13]:
tickets = list(range(1, 31))

In [14]:
student = list(range(1, 21))

In [15]:
shuffle(tickets)

In [ ]:
n_param = 100000
student = list(range(1, 21))
tickets = list(range(1, 31))
result = []

for _ in range(n_param):
    shuffle(tickets)
    result.append(tickets[0] in student)

In [17]:
np.mean(result)

0.66718

In [18]:
n_param = 100000
student = list(range(1, 21))
tickets = list(range(1, 31))
result = []

for _ in range(n_param):
    shuffle(tickets)
    result.append(tickets[1] in student)

In [19]:
np.mean(result)  # искажение Байесовского восприятия

0.66654

In [20]:
# Парадокс 3. Синие и зеленые такси


np.random.binomial(1, 0.15)
# 1 - синее
# 0 - зелёное

0

In [4]:
# генератор


def witness(taxi: int) -> int:
    """witness."""
    if np.random.binomial(1, 0.8):
        return taxi
    return abs(taxi - 1)

In [5]:
witness(0)
# input 0 - зелёное, return 1 - синее, ошибся

0

In [ ]:
# nbqa-mypy: disable-all

#  monte carlo
n_param = 100000
result_2: List[Tuple[int, int]] = []
for _ in range(n_param):
    taxi_param = np.random.binomial(1, 0.15)
    witness_answer_1 = witness(taxi_param)
    result_2.append((taxi_param, witness_answer_1))

In [11]:
t_data_2 = pd.DataFrame(result_2, columns=["taxi", "witness_answer"])
t_data_2

,taxi,witness_answer
0,0,0
1,0,0
2,1,1
3,0,0
4,0,0
...,...,...
99995,0,0
99996,0,0
99997,0,0
99998,0,0


In [9]:
t_data_2.groupby("witness_answer")["taxi"].mean()
# 1    0.416681 41% случаев свидетель будет прав, что реально принадлежит синим
# самое интересное что свидетель верно определяет в 80% случаев,
# получается это опять искажение Баейсовского восприятия

witness_answer
0    0.043177
1    0.416356
Name: taxi, dtype: float64

In [ ]:
# Парадокс 4. Русская рулетка


chamber = [1, 1, 0, 0, 0, 0]

In [15]:
# pylint: disable=redefined-outer-name


def one_turn(chamber_p: List[int]) -> List[int]:
    """One turn."""
    new_chamber = [0, 0, 0, 0, 0, 0]
    n_param = len(chamber_p)
    for i in range(n_param):
        if i < n_param - 1:
            new_chamber[i + 1] = chamber_p[i]
        else:
            new_chamber[0] = chamber_p[i]
    return new_chamber

In [13]:
# pylint: disable=redefined-outer-name


def spin_chamber(chamber_p: List[int]) -> List[int]:
    """spin_chamber."""
    n_param = np.random.randint(1, 7)
    for _ in range(n_param):
        chamber_p = one_turn(chamber_p)
    return chamber_p

In [16]:
spin_chamber([1, 1, 0, 0, 0, 0])

[0, 1, 1, 0, 0, 0]

In [17]:
pd.Series([spin_chamber([1, 1, 0, 0, 0, 0]) for _ in range(100000)]).astype(
    str
).value_counts(normalize=True)

[0, 1, 1, 0, 0, 0]    0.16759
[0, 0, 1, 1, 0, 0]    0.16735
[0, 0, 0, 1, 1, 0]    0.16732
[0, 0, 0, 0, 1, 1]    0.16718
[1, 0, 0, 0, 0, 1]    0.16698
[1, 1, 0, 0, 0, 0]    0.16358
Name: proportion, dtype: float64

In [19]:
n_param = 10000
result_3: List[Tuple[int, int]] = []

for _ in range(n_param):
    # заряжаем револьвер
    chamber = [1, 1, 0, 0, 0, 0]
    # первый игрок крутит барабан
    chamber = spin_chamber(chamber)
    # первый игрок стреляет
    chamber = one_turn(chamber)
    p1 = chamber[0]
    # убрали пулю после выстрела
    chamber[0] = 0

    # второй игрок вращает барабан -- нужно ли это действие?
    chamber = spin_chamber(chamber)

    # второй игрок стреляет
    chamber = one_turn(chamber)
    p2 = chamber[0]

    result_3.append((p1, p2))

In [24]:
t_data_2 = pd.DataFrame(result_3, columns=["p1", "p2"])
t_data_2

,p1,p2
0,0,0
1,1,0
2,0,0
3,0,0
4,1,0
...,...,...
9995,0,1
9996,0,0
9997,1,0
9998,0,0


In [ ]:
t_data_2.groupby("p1")["p2"].agg(["count", "mean"])
# 0 - выжил, первый игрок выжил, вероятность что 2 игрок умрет,
# если крутонешь барабан 0.33%
# 1 - умер

,count,mean
p1,,
0,6588,0.336066
1,3412,0.165006
